# CS405 Machine Learning: Lab 2 Preliminary
### Name: 车凯威
### ID: 12032207

__Objectives__：Text mining (deriving information from text) is a wide field which has
gained popularity with the huge text data being generated. Automation of a
number of applications like sentiment analysis, document classification, topic classification, text summarization, machine translation, etc., has been
done using machine learning models. In this lab, you are required to write
your spam filter by using naïve Bayes method. This time you should not
use 3
rd party libraries including scikit-learn.

## Instruction:
Spam filtering is a beginner’s example of document classification task
which involves classifying an email as spam or non-spam (a.k.a. ham) mail. Email dataset will be provided. We will walk through the following steps
to build this application:  
1) Preparing the text data  
2) Creating word dictionary  
3) Feature extraction process  
4) Training the classifier  
5) Checking the results on test set  

## 1 Preparing the text data:
The data-set used here, is split into a training set and a test set containing
702 mails and 260 mails respectively, divided equally between spam and
ham mails. You will easily recognize spam mails as it contains *spmsg*
in its filename.

In any text mining problem, text cleaning is the first step where we
remove those words from the document which may not contribute to the
information we want to extract. Emails may contain a lot of undesirable
characters like punctuation marks, stop words, digits, etc which may not
be helpful in detecting the spam email. The emails in Ling-spam corpus
have been already preprocessed in the following ways:  

a) Removal of stop words – Stop words like “and”, “the”, “of”, etc are
very common in all English sentences and are not very meaningful in
deciding spam or legitimate status, so these words have been removed
from the emails.   

b) Lemmatization – It is the process of grouping together the different
inflected forms of a word so they can be analysed as a single item. For
example, “include”, “includes,” and “included” would all be
represented as “include”. The context of the sentence is also preserved
in lemmatization as opposed to stemming (another buzz word in text
mining which does not consider meaning of the sentence)  

We still need to remove the non-words like punctuation marks or special
characters from the mail documents. There are several ways to do it. Here, we will remove such words after creating a dictionary, which is a very
convenient method to do so since when you have a dictionary; you need
to remove every such word only once.

In [1]:
import os
import numpy as np
from collections import Counter

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix



def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]    
    all_words = []       
    for mail in emails:    
        with open(mail) as m:
            for i,line in enumerate(m):
                if i == 2:
                    words = line.split()
                    all_words += words
    
    dictionary = Counter(all_words)
    
    list_to_remove = list(dictionary)
    for item in list_to_remove:
        if item.isalpha() == False: 
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
    dictionary = dictionary.most_common(3000)
    return dictionary
    
def extract_features(mail_dir): 
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files),3000))
    docID = 0
    for fil in files:
      with open(fil) as fi:
        for i,line in enumerate(fi):
          if i == 2:
            words = line.split()
            for word in words:
              wordID = 0
              for i,d in enumerate(dictionary):
                if d[0] == word:
                  wordID = i
                  features_matrix[docID,wordID] = words.count(word)
        docID = docID + 1     
    return features_matrix
    
# Create a dictionary of words with its frequency
train_dir = 'ling-spam\\train-mails'
dictionary = make_Dictionary(train_dir)

# Prepare feature vectors per training mail and its labels
train_labels = np.zeros(702)
train_labels[351:701] = 1
train_matrix = extract_features(train_dir)

# Prepare feature vectors per test mail and its labels
test_dir = 'ling-spam\\test-mails'
test_matrix = extract_features(test_dir)
test_labels = np.zeros(260)
test_labels[130:260] = 1

In [2]:
P_true = 350/702
P_false = 1-P_true

In [47]:
def train_algorithm(train_matrix):
        
    # true_features_num =  np.sum((train_matrix[351:701]),axis=0)+1
    # true_num = np.sum(train_matrix[351:701])+2

    # false_features_num =  np.sum((train_matrix[0:350]),axis=0)+1
    # false_num = np.sum(train_matrix[0:350])+2

    # p_feature_true = np.log(true_features_num/true_num)
    # p_feature_false = np.log(false_features_num/false_num)
        


    p0Num = np.ones(3000)
    p1Num = np.ones(3000)
    p0Denom = 2.0
    p1Denom = 2.0

    for i in range(350):
        p0Num += train_matrix[i]
        p0Denom += sum(train_matrix[i])

    for i in range(351,702):
        p1Num += train_matrix[i]
        p1Denom += sum(train_matrix[i])

    p_feature_true = np.log(p1Num / p1Denom)
    p_feature_false = np.log(p0Num / p0Denom)

    return p_feature_true,p_feature_false

In [44]:
p0Num = np.ones(3000)
p1Num = np.ones(3000)
p0Denom = 2.0
p1Denom = 2.0


for i in range(350):
    p0Num += train_matrix[i]
    p0Denom += sum(train_matrix[i])

for i in range(351,702):
    p1Num += train_matrix[i]
    p1Denom += sum(train_matrix[i])


p_feature_true = np.log(p1Num / p1Denom)
p_feature_false = np.log(p0Num / p0Denom)

In [42]:
x1 = np.array([[1,2],[3,4])
x2 = np.array([3,4])

x1 * x2
x3 = test_matrix[1,:]+1
x3 * p_feature_true

array([-8.60262595, -8.89284607, -4.40386342, ..., -9.5278274 ,
       -9.39429601, -9.5278274 ])

In [56]:
x1 = np.array([[1,2,3],[3,4,5]])
x2 = np.array([3,4,8])
(x1+1) * x2

array([[ 6, 12, 32],
       [12, 20, 48]])

In [78]:
def train_algorithm(train_matrix):
        
    # true_features_num =  np.sum((train_matrix[351:701]),axis=0)+1
    # true_num = np.sum(train_matrix[351:701])+2

    # false_features_num =  np.sum((train_matrix[0:350]),axis=0)+1
    # false_num = np.sum(train_matrix[0:350])+2

    # p_feature_true = np.log(true_features_num/true_num)
    # p_feature_false = np.log(false_features_num/false_num)
        


    p0Num = np.ones(3000)
    p1Num = np.ones(3000)
    p0Denom = 2.0
    p1Denom = 2.0

    for i in range(350):
        p0Num += train_matrix[i]
        p0Denom += sum(train_matrix[i])

    for i in range(351,702):
        p1Num += train_matrix[i]
        p1Denom += sum(train_matrix[i])

    p_feature_true = np.log(p1Num / p1Denom)
    p_feature_false = np.log(p0Num / p0Denom)

    return p_feature_true,p_feature_false


def pred(test_matrix,p_feature_true,p_feature_false):

    p_1 = []
    p_0 = []
    result = []
    test_matrix2 = np.copy(test_matrix)
    #test_matrix2[test_matrix2 == 0] = 1
    for i in range(260):
        p_1.append(np.sum(test_matrix2[i] * p_feature_true) + np.log(P_true))
        p_0.append(np.sum(test_matrix2[i] * p_feature_false) + np.log(P_false))
   # print(p_1)
    #print(p_0)
    # p_1 = np.sum((test_matrix+1) * p_feature_true, axis = 1) + np.log(P_true)
    # p_0 = np.sum((test_matrix+1) * p_feature_true, axis = 1) + np.log(P_false)

    for i in range(260):
      #  print("p1" + str(p_1[i]))
        
       # print("p0" + str(p_0[i])+"\n")
        if p_1[i] > p_0[i]:
            result.append(1)
        else:
            result.append(0)

    return result

    #print(prb_false)

p_feature_true,p_feature_false = train_algorithm(train_matrix)

result = pred(test_matrix,p_feature_true,p_feature_false)

correct = 0
for i,item in enumerate(result):
    if result[i]==test_labels[i]:
        correct+=1
correct
acc = correct/260
acc

0.9615384615384616

In [74]:
list(result).count(0)

138

In [65]:
import math
def pred(test_matrix,p_feature_true,p_feature_false):

    p_1 = []
    p_0 = []
    result = []
    test_matrix2 = np.copy(test_matrix)
    #test_matrix2[test_matrix2 == 0] = 1
    for i in range(260):

        p_1.append(np.sum((test_matrix2[i,:]+1) * p_feature_true) + np.log(P_true))
        p_0.append(np.sum((test_matrix2[i,:]+1) * p_feature_false) + np.log(P_false))

    # p_1 = np.sum((test_matrix+1) * p_feature_true, axis = 1) + np.log(P_true)
    # p_0 = np.sum((test_matrix+1) * p_feature_true, axis = 1) + np.log(P_false)

    for i in range(260):
        if p_1[i] > p_0[i]:
            result.append(1)
        else:
            result.append(0)

    return result

    #print(prb_false)

p_feature_true,p_feature_false = train_algorithm(train_matrix)

result = pred(test_matrix,p_feature_true,p_feature_false)

correct = 0
for i,item in enumerate(result):
    if result[i]==test_labels[i]:
        correct+=1
correct
acc = correct/260
acc

0.5076923076923077

In [68]:
p_feature_true
p_feature_false

array([ -6.21434021,  -5.8088751 ,  -7.19516946, ...,  -9.83422679,
       -10.2396919 ,  -9.83422679])